In [ ]:
import sys

from active_critic.model_src.transformer import ModelSetup, TransformerModel, generate_square_subsequent_mask
from active_critic.model_src.base_transformer import DebugTEL
import torch as th
from active_critic.utils.pytorch_utils import calcMSE

In [ ]:
from active_critic.model_src.whole_sequence_model import WholeSequenceModel
from active_critic.model_src.transformer import ModelSetup, TransformerModel
import torch as th
from active_critic.utils.test_utils import make_mask_data, make_seq_encoding_data, make_critic_data, make_wsm_setup
import unittest

In [ ]:
th.manual_seed(0)
device = 'cuda'
seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3

wsa_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output)

wsa = WholeSequenceModel(wsms=wsa_setup)
input = th.ones([batch_size, seq_len, d_intput],
                dtype=th.float, device=device)
output = wsa.forward(inputs=input)
shape = output.shape


inpt_seq, outpt_seq = make_seq_encoding_data(
    batch_size=batch_size, seq_len=seq_len, ntoken=ntoken, d_out=d_output)
success = th.ones_like(inpt_seq, dtype=th.bool)
wsa = WholeSequenceModel(wsms=wsa_setup)
data = inpt_seq, outpt_seq, success

obsvs = th.zeros([batch_size, seq_len, 3], dtype=th.float32, device=device)
obsvs[1] = 1
acts = th.rand([batch_size, seq_len, 2], dtype=th.float32, device=device)

In [ ]:
actor = WholeSequenceModel(wsa_setup)
for i in range(3000):
    res = actor.optimizer_step(inputs=obsvs, label=acts)
    if i%100==0:
        print(res)

In [5]:
actor.optimizer.state_dict()

{'state': {0: {'step': tensor(626.),
   'exp_avg': tensor([[ 7.4847e-05,  1.9298e-05,  4.2887e-05, -9.3536e-07,  7.9733e-06,
             4.4446e-06,  5.4465e-06,  8.0966e-06,  7.0321e-06, -1.4133e-06],
           [ 9.5993e-06,  8.6455e-07,  3.9063e-06, -9.0849e-06,  3.0903e-06,
            -1.9915e-09, -1.1711e-06, -4.2359e-06, -2.7055e-06, -1.6443e-05],
           [-7.0292e-05, -2.8123e-05, -5.3616e-05, -7.0056e-05,  8.3656e-06,
            -9.2167e-06, -2.0217e-05, -4.9647e-05, -3.5532e-05, -1.2497e-04],
           [-3.8958e-05, -1.7302e-05, -3.4913e-05, -6.1970e-05,  9.5844e-06,
            -6.9375e-06, -1.6294e-05, -4.1453e-05, -2.9342e-05, -1.1001e-04],
           [ 1.0394e-05,  2.8511e-06,  1.6972e-06, -1.7084e-05,  4.4831e-06,
            -9.6812e-07, -3.1277e-06, -9.2528e-06, -6.2881e-06, -2.8994e-05],
           [-1.2937e-04, -3.7618e-05, -7.3823e-05, -4.3698e-06, -1.1893e-05,
            -7.6012e-06, -1.0322e-05, -1.7104e-05, -1.4128e-05, -1.0065e-05],
           [ 4.0491e-0

In [ ]:
for i in range(3000):
    res = wsa.optimizer_step(inputs=obsvs, label=acts)
    if i%100==0:
        print(res)

In [1]:
from active_critic.utils.pytorch_utils import get_steps_from_actions, pick_action_from_history
import torch as th

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
actions = th.arange(2*3*3*4).reshape([2, 3, 3, 4])
steps = get_steps_from_actions(actions=actions)

In [3]:
steps

tensor([[0, 1, 2],
        [0, 1, 2]])

In [4]:
actions

tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],

         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]],

         [[24, 25, 26, 27],
          [28, 29, 30, 31],
          [32, 33, 34, 35]]],


        [[[36, 37, 38, 39],
          [40, 41, 42, 43],
          [44, 45, 46, 47]],

         [[48, 49, 50, 51],
          [52, 53, 54, 55],
          [56, 57, 58, 59]],

         [[60, 61, 62, 63],
          [64, 65, 66, 67],
          [68, 69, 70, 71]]]])

In [5]:
res = pick_action_from_history(action_histories=actions, steps=steps)

In [6]:
res

tensor([[[ 0.,  1.,  2.,  3.],
         [16., 17., 18., 19.],
         [32., 33., 34., 35.]],

        [[36., 37., 38., 39.],
         [52., 53., 54., 55.],
         [68., 69., 70., 71.]]])

In [ ]:
wsa.init_model()
res = wsa.optimizer_step(inputs=obsvs, label=outpt_seq)

for i in range(3000):
    res = wsa.optimizer_step(inputs=obsvs, label=outpt_seq)

In [ ]:
ms = ModelSetup()
seq_len = 3
ntoken = 3
batch_size = 2
d_output = 4

ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 1
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cpu'
device = 'cpu'


In [ ]:
def make_seq_encoding_data(batch_size, seq_len, ntoken, d_out, device = 'cuda'):
    inpt_seq = th.ones([batch_size,seq_len,ntoken], dtype=th.float, device=device)
    outpt_seq = th.ones([batch_size,seq_len,d_out], dtype=th.float, device=device)
    outpt_seq[:,::2] = 0
    return inpt_seq, outpt_seq

In [ ]:
tm = TransformerModel(model_setup=ms)
mask = generate_square_subsequent_mask(3)
mask = mask.unsqueeze(0).repeat([2, 1, 1])
mask[0] = 0
#mask.shape

In [ ]:
def pull_tens_to_front(src, i):
    if i > 0:
        src[i, :, :-i] = src[i,:,i:]
        src[i, :, -i:] = -1

    return src

def pull_tens_to_front_sparse(src, i):
    pulled = src[i,:,i:]
    max_pulled = pulled.max(dim=-2).values == 1
    max_pulled = max_pulled.unsqueeze(-2).repeat([1, 1, src.shape[2] - i, 1])
    if i > 0:
        src[i, :, :-i] = max_pulled
        src[i, :, -i:] = -1
    else:
        src[i, :] = max_pulled

    return src
def repeat_along_seq(src):
    src = src.repeat([src.shape[1], 1, 1]).reshape([src.shape[1], src.shape[0], src.shape[1], src.shape[2]])
    return src


In [ ]:
obsv = th.rand([2,3,4], requires_grad=True)
act = th.rand([2,3,2])
rewards = th.rand([2,3,1])

def make_seq_encoding_data(actions, obsv, rewards):
    actions = repeat_along_seq(actions)
    obsv = repeat_along_seq(obsv)
    rewards = repeat_along_seq(rewards)
    for i in range(len(obsv)):
        obsv[i] = obsv[i,:,i].unsqueeze(1)

        actions = pull_tens_to_front(actions, i)
        rewards = pull_tens_to_front_sparse(rewards, i)
    return actions, obsv, rewards

In [ ]:
rewards[0,0,0] = 1
a, o, r = make_seq_encoding_data(actions=act, obsv=obsv, rewards=rewards)

In [ ]:
a

In [ ]:
obsv

In [ ]:
obsv

In [ ]:
obsv

In [ ]:
def make_partial_sequence(seq):
    

In [ ]:
result = tm.forward(src=obsv, mask=mask)
loss = ((result[:, -2])**2).mean()
loss.backward()


In [ ]:
obsv.grad

In [ ]:
result

In [ ]:
def make_mask_data(batch_size, seq_len, ntoken, device = 'cuda'):
    mask = generate_square_subsequent_mask(seq_len).to(device)
    inpt_seq = th.ones([batch_size,seq_len,ntoken], dtype=th.float, device=device)
    inpt_seq[0,-1,0] = 0
    outpt_seq = th.ones_like(inpt_seq)
    outpt_seq[0] = 0
    return inpt_seq, outpt_seq, mask

In [ ]:
inpt_seq, outpt_seq = make_seq_encoding_data(batch_size, seq_len, ntoken, d_out = d_output, device=device)

In [ ]:
def make_no_conflict_part_data(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, 1:] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0] += 1

    res, actions = make_part_observed(inpt, actions)
    return res, actions
    

In [ ]:
def make_part_observed(inpt, actions):
    #batch, seq, dim
    inpt = inpt.unsqueeze(1).repeat([1,inpt.shape[1], 1, 1])
    inpt = inpt.permute([0,3,1,2])
    res = th.triu(inpt)
    res = res.permute([0,2,3,1]).reshape([-1,seq_len, obs_dim])
    
    rep_actions = actions.repeat([1,seq_len,1]).reshape([-1,seq_len, d_output])
    return res, rep_actions

In [ ]:
def make_conflict_part_data(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, :1] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0, :1] += 1

    res, actions = make_part_observed(inpt, actions)
    rew = th.ones([batch_size * seq_len, seq_len], device = device)
    
    return res, actions, rew

In [ ]:
def make_conflict_part_data_neg(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, -1:] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0, -1:] += 1

    res, actions = make_part_observed(inpt, actions)
    rew = th.ones([batch_size * seq_len, seq_len], device = device)
    scale = th.arange(seq_len).reshape([1,-1]).repeat([rew.shape[0], 1])
    rew = rew * scale / (seq_len - 1)
    rew[:,-1:] = 0
    
    res_copy = res.clone()
    res[:seq_len] = res_copy[seq_len:]
    res[seq_len:] = res_copy[:seq_len]
    
    return res, actions, rew

In [ ]:
def make_sequence(obs, acts):
    result = th.cat((obs, acts), dim=-1)
    return result

In [ ]:
obs_dim = 3


In [ ]:
obs_nc, act_nc = make_no_conflict_part_data(batch_size, seq_len, obs_dim, d_output, device='cpu')
obs_c, act_c, rew_c_pos = make_conflict_part_data(batch_size, seq_len, obs_dim, d_output, device='cpu')
obs_c_n, act_c_n, rew_c_neg = make_conflict_part_data_neg(batch_size, seq_len, obs_dim, d_output, device='cpu')

In [ ]:
rew_c_neg

In [ ]:
seq_n = make_sequence(obs_c_n,act_c_n)
seq_p = make_sequence(obs_c,act_c)

In [ ]:
seq = th.cat((seq_n, seq_p), dim=0)
rew = th.cat((rew_c_pos, rew_c_neg), dim=0)

In [ ]:
ms = ModelSetup()
seq_len = 3
ntoken = 3 + 4
batch_size = 2
d_output = 1

ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 1
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cpu'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.model_class:TransformerModel = TransformerModel
device = 'cpu'


In [ ]:
inpt = seq

model = TransformerModel(model_setup=ms).to(device)
with th.no_grad():
    answer = model.forward(inpt)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(3000):
    result = model.forward(inpt)
    loss = calcMSE(result, rew)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss

In [ ]:
inpt[0]

In [ ]:
answer = model.forward(inpt[5].unsqueeze(0))

In [ ]:
answer

In [ ]:
rew[5]

In [ ]:
inpt[9]

In [ ]:
result, attention = model.forward(res_c[0].unsqueeze(0), return_attention=True)

In [ ]:
result

In [ ]:
attention

In [ ]:
result, attention = model.forward(res[4].unsqueeze(0), return_attention=True)

In [ ]:
attention

In [ ]:
res[4]

In [ ]:
mask = generate_square_subsequent_mask(seq_len).to('cuda')

In [ ]:
inpt_seq = th.ones([2,seq_len,1], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones_like(inpt_seq)
outpt_seq[0] = 0

In [ ]:
inpt_seq, outpt_seq, mask = make_mask_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken)


In [ ]:
model = TransformerModel(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(1000):
    result = model.forward(inpt_seq, mask=None)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss

In [ ]:
from ActiveCritic.model_src.transformer import CriticTransformer

In [ ]:
ms = ModelSetup()
seq_len = 6
d_output = 1
ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 2
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cuda'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.d_result = 1
ms.model_class:TransformerModel = CriticTransformer

In [ ]:
inpt_seq = th.ones([2,seq_len,4], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones([2,1], dtype=th.float, device='cuda')
outpt_seq[0] = 0

In [ ]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    a = model.forward(inpt_seq)

In [ ]:
a.shape

In [ ]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(2000):
    result = model.forward(inpt_seq)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss